# LLM + RAG

### Importações

In [1]:
import dotenv
import os
import numpy as np

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.vectorstores import Chroma
import chromadb
import pprint

dotenv.load_dotenv()

True

### Embedding Model

In [3]:
embedding = OpenAIEmbeddings(model=os.getenv('MODEL_NAME'), embedding_ctx_length=10000)

### Retriever

In [4]:
c_client = chromadb.HttpClient('0.0.0.0',8000)
c_client.list_collections()

[Collection(name=anpd)]

In [5]:
anpd_data = c_client.get_collection(os.getenv('COLLECTION_NAME'))
anpd_data.count()

539

In [10]:
idx = 3

sample = anpd_data.peek()
print("########## TEXTO ##########")
pprint.pp(sample['documents'][idx])
print("\n########## ID ##########")
pprint.pp(sample['ids'][idx])
print("\n########## EMBEDDING ##########")
pprint.pp(np.array(sample['embeddings'][idx]))
pprint.pp(np.array(sample['embeddings'][idx]).shape)
print("\n########## METADATA ##########")
pprint.pp(sample['metadatas'][idx])

########## TEXTO ##########
('Prevenção à fraude e à segurança | o modelo de teste de balanceamento '
 'também \n'
 'pode ser utilizado no caso de tratamento baseado na hipótese legal prevista '
 'no art. \n'
 '11, ii, g, da lgpd, como forma de avaliar se, no caso concreto, prevalecem '
 'direitos e li-\n'
 'berdades fundamentais dos titulares. Em especial, deve-se considerar que '
 'essa hipó-\n'
 'tese legal é aplicável exclusivamente para fins de “prevenção à fraude e à '
 'segurança \n'
 'do titular , nos processos de identificação e autenticação de cadastro em '
 'sistemas ele-\n'
 'trônicos”. Esta finalidade deve ser interpretada restritivamente e descrita '
 'de forma \n'
 'objetiva e o mais detalhada possível.\n'
 'Dados de crianças e adolescentes | caso o tratamento envolva dados pessoais '
 'de \n'
 'crianças e adolescentes, o melhor interesse dos titulares deve ser avaliado '
 'de forma \n'
 'prioritária em todas as fases do teste, prevalecendo a interpretação que '
 'atenda

In [11]:
vectordb = Chroma(collection_name = os.getenv('COLLECTION_NAME'), 
                  embedding_function=embedding,
                  client=c_client)

In [12]:
## criando o retriever

retriever = vectordb.as_retriever(search_type='similarity',
                                 search_kwargs={'k':3})

In [13]:
## testando o retriever

contexto = retriever.get_relevant_documents("O que são cookies e quais suas categorias?")
len(contexto)

3

In [14]:
contexto

[Document(page_content='9\nguia anpd · Cookies e proteção de dados pessoaismentais. Neste último caso, é possível considerar o perfil comporta-\nmental como um dado pessoal, uma vez que associado a uma pessoa \nnatural[ 2 ].  Nesse sentido, nas hipóteses acima mencionadas, as infor -\nmações pessoais coletadas por meio de cookies podem ser considera-\ndas dados pessoais, cujo tratamento é regulado pela lgpd.\nCategorias de cookies\nAs categorias para definição dos cookies são diversas e podem partir de \ndiferentes perspectivas. No presente Guia, serão apresentadas algu-\nmas das categorias mais comumente utilizadas, de forma não exausti-\nva, organizadas pelos tipos de cookies mais usuais. Importante conside-\nrar que um mesmo cookie pode ser incluído em mais de uma categoria.\nDesse modo, a seguir são abordadas as categorias de cookies de acor -\ndo com: (i) a entidade responsável pela sua gestão; (ii) a necessidade; \n(iii) a finalidade; e (iv) o período de retenção das informações.

## Augment and Generate

In [36]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# query_template = """Seu trabalho é montar um plano de ação dado a situação apresentada no contexto.
# Utilize o máximo de conhecimento possível do contexto, caso não tenha certeza de como estruturar esse plano
# diga que não sabe. Esse plano de ação para resolução do problema descrito na pergunta deve ter
# passos e subpassos e indicar quais documentos precisam ser lidos para resolver o  problema. Não
# crie passos e subpassos ligados ao conhecimento fora do contexto. Suas resposta devem ser sempre 
# em português do Brasil, utilizando linguagem formal e de simples entendimento para o usário, 
# quando possível dê exemplos para auxiliar o entendimento dos conceitos provenientes.

# Contexto : {contexto}
# Pergunta : {pergunta}

# Resposta: 
# """
query_template = """Não gere nenhum FAQ sobre assuntos fora do contexto.
Gere uma FAQ sobre o contexto abaixo utilizando como referência também o tema.
Gere no máximo 10 perguntas e respostas.
A FAQ deve seguir a estrutura abaixo:

<NUMERO DA PERGUNTA>. <PERGUNTA>
Respota: <RESPOSTA DA PERGUNTA>

{contexto}
{tema}

"""


template = PromptTemplate.from_template(query_template)

In [37]:
llm = ChatOpenAI(model="gpt-4",
                temperature=0.3)

In [40]:
rag_chain = (
    {"contexto": retriever | format_docs, "tema": RunnablePassthrough()}
    | template
    | llm
    | StrOutputParser()
)

In [41]:
print(
    rag_chain.invoke("""Tratamento de cookies na LGPD.""")
)

1. O que são os princípios da finalidade, necessidade e adequação na LGPD?
Resposta: São princípios que determinam que a coleta de dados pessoais deve ser limitada ao mínimo necessário para a realização de finalidades legítimas, explícitas e específicas, e que o tratamento posterior desses dados não pode ser incompatível com essas finalidades.

2. Como esses princípios se aplicam à coleta de dados por meio de cookies?
Resposta: A utilização de cookies deve ser específica e informada ao titular, e a coleta de dados deve ser compatível com a finalidade informada. Por exemplo, se o responsável pela página eletrônica informa ao titular que utiliza cookies apenas para medição de audiência, não poderá utilizar as informações coletadas para outras finalidades.

3. O que a LGPD diz sobre o uso de cookies não necessários?
Resposta: O consentimento do titular será mais apropriado quando a coleta de informações for realizada por cookies não necessários, ou seja, aqueles que não são essenciais par

In [44]:
print(
    rag_chain.invoke(
        """Sou um laboratório de análises clínicas e gostaria de recolher e utilizar
        os resultados dos exames de meus clientes para pesquisas. O que preciso fazer para
        me proteger e também ao meu cliente?"""
    )
)

Para realizar pesquisas com os resultados dos exames de seus clientes, é necessário seguir algumas diretrizes para garantir a proteção dos dados pessoais. Primeiramente, é importante que você obtenha o consentimento dos titulares dos dados para o uso das informações na pesquisa. Além disso, é essencial garantir a confidencialidade e a privacidade dos dados, o que pode ser feito através de medidas como a anonimização e a pseudonimização. 

De acordo com a Norma Operacional nº 001/2013, do Conselho Nacional de Saúde, deve ser apresentado um "termo de compromisso" assinado pelo "responsável maior da instituição". Este termo deve dispor sobre a finalidade e a destinação que fundamentaram sua autorização, bem como as obrigações a que se submeterá o requerente.

Além disso, é importante respeitar o princípio da boa-fé, que deve orientar todas as atividades de tratamento de dados pessoais. Isso significa que os dados devem ser utilizados apenas para a finalidade para a qual foram coletados e 

In [26]:
print(
    rag_chain.invoke(
        """Sou um cliente do Banco do Brasil, quais são os dados que eles coletam sobre mim?"""
    )
)

Não tenho informações específicas sobre quais dados o Banco do Brasil coleta sobre seus clientes, pois meu treinamento é baseado em guias orientativos da ANPD (Autoridade Nacional de Proteção de Dados). No entanto, em geral, bancos costumam coletar dados como nome, CPF, endereço, telefone, e-mail, renda, informações de emprego, entre outros, para fins de identificação, análise de crédito, prevenção à fraude e cumprimento de obrigações legais. Recomendo que você entre em contato diretamente com o Banco do Brasil ou consulte a política de privacidade deles para obter informações mais precisas.


In [27]:
## trazendo as fontes utilizadas para a resposta

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["contexto"])))
    | template
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"contexto": retriever, "pergunta": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [28]:
pprint.pp(
    rag_chain_with_source.invoke("O que são cookies e quais são suas categorias?")
)

{'contexto': [Document(page_content='9\nguia anpd · Cookies e proteção de dados pessoaismentais. Neste último caso, é possível considerar o perfil comporta-\nmental como um dado pessoal, uma vez que associado a uma pessoa \nnatural[ 2 ].  Nesse sentido, nas hipóteses acima mencionadas, as infor -\nmações pessoais coletadas por meio de cookies podem ser considera-\ndas dados pessoais, cujo tratamento é regulado pela lgpd.\nCategorias de cookies\nAs categorias para definição dos cookies são diversas e podem partir de \ndiferentes perspectivas. No presente Guia, serão apresentadas algu-\nmas das categorias mais comumente utilizadas, de forma não exausti-\nva, organizadas pelos tipos de cookies mais usuais. Importante conside-\nrar que um mesmo cookie pode ser incluído em mais de uma categoria.\nDesse modo, a seguir são abordadas as categorias de cookies de acor -\ndo com: (i) a entidade responsável pela sua gestão; (ii) a necessidade; \n(iii) a finalidade; e (iv) o período de retenção das

In [29]:
pprint.pp(
    rag_chain_with_source.invoke(
        """Sou um laboratório de análises clínicas e gostaria de recolher utilizar
        os resultados dos exames de meus clientes para pesquisas. O que preciso fazer para
        me proteger e também ao meu cliente?"""
    )
)

{'contexto': [Document(page_content='48\nguia anpd · Tratamento de dados pessoais para fins acadêmicos e para a realização de estudos e pesquisasdos dados para fins de realização da pesquisa e à atribuição de res-\nponsabilidades para os pesquisadores e instituições de pesquisa.\nDe fato, a posição dos pesquisadores nessas situações pode ser equi-\nparada à de profissionais que têm o dever de conferir sigilo às infor -\nmações recebidas no exercício de sua atividade profissional, a exem-\nplo de médicos e advogados. Conforme as orientações fornecidas pela \nComissão Nacional de Ética em Pesquisa, os pesquisadores devem \nassegurar aos participantes “o compromisso profissional com o sigilo abso-\nluto das informações” utilizadas na pesquisa[ 4 ].\nO Conselho Nacional de Saúde publicou a Cartilha dos \ndireitos dos participantes de pesquisa.\nClique aqui e acesse a Cartilha.\nTrata-se, ademais, de uma decorrência do princípio da boa-fé, que \ndeve orientar todas as atividades de tratamen

In [30]:
pprint.pp(
    rag_chain_with_source.invoke(
        """Sou um cliente do Banco do Brasil, quais são os dados que eles coletam sobre mim?"""
    )
)

{'contexto': [Document(page_content='blico municipal identificou que o Departamento de Recursos Humanos \nsolicita de novos servidores o preenchimento de uma ficha cadastral e \nde um formulário denominado “perfil socioeconômico” . Deste último, \nconstam dados sensíveis, tais como informações sobre origem racial, \nconvicção religiosa e filiação a sindicato. A equipe verificou que o for -\nmulário foi utilizado há alguns anos para subsidiar uma pesquisa efe-\ntuada em convênio com uma instituição de ensino local. Porém, mesmo \napós o fim do convênio, o formulário continuou sendo utilizado como \nparte do cadastro de novos servidores. Além disso, foi identificado que \nnão é atribuída restrição de acesso aos processos administrativos nos \nquais estas e outras informações funcionais dos servidores são anexa-\ndas. A justificativa apresentada para tanto foi a de que o nome do servi-\ndor é excluído dos documentos tornados públicos. No entanto, a equipe \nconstatou que a reindentificaçã